# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input
from tensorflow.keras.callbacks import EarlyStopping
import optuna
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima
import os
import holidays


#Save requirements
os.system("pip freeze > requirements.txt")

#SEED   
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
df = pd.read_csv('Data/london_merged.csv')
display(df.head(2))

- Metadata:
  - "timestamp" - timestamp field for grouping the data
  - "cnt" - the count of a new bike shares
  - "t1" - real temperature in C
  - "t2" - temperature in C "feels like"
  - "hum" - humidity in percentage
  - "wind_speed" - wind speed in km/h
  - "weather_code" - category of the weather
  - "is_holiday" - boolean field - 1 holiday / 0 non holiday
  - "is_weekend" - boolean field - 1 if the day is weekend
  - "season" - category field meteorological seasons: 0-spring ; 1-summer; 2-fall; 3-winter.
  - "weathe_code" category description:
     - 1 = Clear ; mostly clear but have some values with haze/fog/patches of fog/ fog in vicinity 
     - 2 = scattered clouds / few clouds 
     - 3 = Broken clouds 
     - 4 = Cloudy 
     - 7 = Rain/ light Rain shower/ Light rain 
     - 10 = rain with thunderstorm 
     - 26 = snowfall 
     - 94 = Freezing Fog

In [ ]:
df.info()

# Data Wrangling

In [ ]:
#Convert the timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
#Keep records from 2015 and 2016
df = df[(df['timestamp'].dt.year == 2015) | (df['timestamp'].dt.year == 2016)]
#Sort the values by timestamp
df = df.sort_values('timestamp')

In [ ]:
#Missing values
df.isnull().sum()

- No missing values. But there might be missing timestamps.

In [ ]:
#Check for missing timestamps
all_days = pd.date_range(start=df['timestamp'].min(), end=df['timestamp'].max(), freq='h')
missing_days = all_days[~all_days.isin(df['timestamp'])]
print('Number of missing timestamps:', len(missing_days))

In [ ]:
missing_days[0]

- 130 timestamps are missing. We will imput them using existing values.

In [ ]:
#London holidays
uk_holidays = holidays.UK(years=[df['timestamp'].dt.year.min(), df['timestamp'].dt.year.max()])
uk_holidays

In [ ]:
#Create new dataframe using all days
df_full = pd.DataFrame(all_days, columns=['timestamp'])
#Merge with df to get cnt, t1, t2, hum, wind_speed, weather_code, season
df_full = df_full.merge(df[['timestamp', 'cnt', 't1', 't2', 'hum', 'wind_speed', 'weather_code', 'season']], on='timestamp', how='left')
#is_holiday column: 1 if holiday, 0 if not
df_full['is_holiday'] = np.where(df_full['timestamp'].dt.date.isin(uk_holidays), 1, 0)
df_full['is_weekend'] = np.where(df_full['timestamp'].dt.dayofweek.isin([5, 6]), 1, 0)

#Backfill missing values
df_full = df_full.ffill()
df = df_full.copy()

In [ ]:
df.isnull().sum()

In [ ]:
missing_days = all_days[~all_days.isin(df['timestamp'])]
print('Number of missing timestamps:', len(missing_days))

In [ ]:
#Set the timestamp as the index
df.set_index('timestamp', inplace=True)
#Set period to 1 hour
df.index = pd.DatetimeIndex(df.index).to_period('h')
df.head(2)

In [ ]:
df = df.resample('D').agg({'cnt':'sum', 
                           't1':'median', 
                           't2':'median', 
                           'hum':'median', 
                           'wind_speed':'median', 
                           'weather_code': lambda x: x.value_counts().index[0], 
                           'season': lambda x: x.value_counts().index[0], 
                           'is_holiday':'max', 
                           'is_weekend':'max'})
df.head(2)

In [ ]:
#Remove duplicates
df.drop_duplicates(inplace=True)

In [ ]:
# # Boxplot of all the columns
plt.figure(figsize=(10, 12))
cols = df.columns
print(cols)
for i in range(1, len(cols)):
    print(cols[i])
    plt.subplot(3, 3, i)
    sns.boxplot(df[cols[i-1]])
    plt.title(cols[i-1])
plt.tight_layout()
plt.show()

- There is no abnormal data in the dataset.

In [ ]:
# Correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.show()

- Real and feels like temperature are highly correlated. Let's use feels like temperature since it is more likely to impact the decision.

In [ ]:
#Drop t1
df.drop('t1', axis=1, inplace=True)

In [ ]:
#Map codes
#Map weather code:
weather_desc = {
    1: 'Clear', 2: 'Scattered_Clouds', 3: 'Broken_Clouds', 4: 'Cloudy', 7: 'Rain', 10: 'Storm', 26: 'Snowfall', 94: 'Freezing_Fog'
}
df['weather_code'] = df['weather_code'].map(weather_desc)

#Map is_holiday:
df['is_holiday'] = df['is_holiday'].map({0:'No_Holiday', 1:'Holiday'})

#Map is_weekend:
df['is_weekend'] = df['is_weekend'].map({0:'Weekday', 1:'Weekend'})

#Map season:
seasons = {0:'Spring', 1:'Summer', 2:'Fall', 3:'Winter'}
df['season'] = df['season'].map(seasons)
df.head(2)

In [ ]:
# #Pairplot
# sns.pairplot(df[['cnt', 't2', 'hum', 'wind_speed', 'weather_code', 'season']],
#              hue='cnt', 
#              palette='coolwarm',
#              height=3,
#              aspect=1.5)
# plt.show()

In [ ]:
#One hot encoding for categorical variables
df = pd.get_dummies(df, drop_first=True, dtype=int)
df.head(2)

# Models

## SARIMAX

### Check Stationarity

In [ ]:
# #Plot to check for stationarity
# plt.figure(figsize=(15, 6))
# plt.plot(y_train.to_timestamp())
# plt.title('Hourly Bike Rentals')
# plt.show()

- There is no noticeable trend in the data. The data looks stationary. Let's verify this with the Augmented Dickey-Fuller test.

In [ ]:
# def check_stationarity(data):
#     print('Null Hypothesis: Presence of unit root (Data is not stationary)')
#     print('Alternate Hypothesis: Absence of unit root (Data is stationary)')
#     result = adfuller(data, autolag='AIC')
#     print(result)
#     print('ADF Statistic:', result[0])
#     print('p-value:', result[1])
#     if result[1] > 0.05:
#         print('Data is not stationary')
#     else:
#         print('Data is stationary')

# #Check stationarity of the target variable
# check_stationarity(y_train)

In [ ]:
# #ACF and PACF plots
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# plt.figure(figsize=(20, 15))
# plt.subplot(211)
# plot_acf(y_train, lags=168, ax=plt.gca())
# plt.subplot(212)
# plot_pacf(y_train, lags=168, ax=plt.gca())
# plt.tight_layout()
# plt.show()

- Estimating AR terms: The PACF plot shows a significant spike at lag 1, 2, 3, 4, 5 and 6. Therefore, p = 6.
- Estimating I term: The ADF test shows that the data is already stationary. Therefore, d = 0.
- Estimating MA terms: The ACF plot shows a significant spike at lag 1, 2, 3, and 4. Therefore, q = 4.
- Estimating seasonal AR terms: The PACF plot shows a significant spike at lag 24. Therefore, P = 1.
- Estimating seasonal I term: The ADF test shows that the data is already stationary. Therefore, D = 0.
- Estimating seasonal MA terms: The ACF plot shows a significant spike at seven 24 lag intervals. Let's start Q with 7.
- ACF plot shows strong correlation at lag 24. This indicates a daily seasonality. Therefore, s = 24.

However, running the model with these parameters is computationally expensive. For this reason, as an example, I will use small order values.

In [ ]:
#Training df till June 2016 and testing df from July 2016
train_df = df.loc[:'2016-06-30']
test_df = df.loc['2016-07-01':]

# Linear Regression

In [ ]:
#Linear regression
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

X_train = train_df.drop('cnt', axis=1)
y_train = train_df['cnt']
X_test = test_df.drop('cnt', axis=1)
y_test = test_df['cnt']

X_train = add_constant(X_train)
X_test = add_constant(X_test)

lr = OLS(y_train, X_train)
lr = lr.fit()

#MAE, RMSE, R2
y_pred = lr.predict(X_test)
mae = np.mean(np.abs(y_test - y_pred))
rmse = np.sqrt(np.mean((y_test - y_pred)**2))
r2 = lr.rsquared
print('MAE:', mae)
print('RMSE:', rmse)
print('R2:', r2)

# XGBoost

In [ ]:
# XGBoost
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

X_train = train_df.drop('cnt', axis=1)
y_train = train_df['cnt']
X_test = test_df.drop('cnt', axis=1)
y_test = test_df['cnt']

model = XGBRegressor(n_estimators=1000, max_depth=10, learning_rate=0.01, random_state=42)
model.fit(X_train, y_train)

#MAE, RMSE, R2
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(np.mean((y_test - y_pred)**2))
r2 = model.score(X_test, y_test)
print('MAE:', mae)
print('RMSE:', rmse)
print('R2:', r2)

# Prophet

In [ ]:
#Prophet
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric

pr_train = train_df.copy()
pr_train = pr_train.reset_index()
pr_train['timestamp'] = pr_train['timestamp'].dt.to_timestamp()
pr_train = pr_train.rename(columns={'timestamp':'ds', 'cnt':'y'})

pr_test = test_df.copy()
pr_test = pr_test.reset_index()
pr_test['timestamp'] = pr_test['timestamp'].dt.to_timestamp()
pr_test = pr_test.rename(columns={'timestamp':'ds', 'cnt':'y'})


model = Prophet()

model.add_regressor('t2')
model.add_regressor('hum')
model.add_regressor('wind_speed')
model.add_regressor('is_holiday_No_Holiday')
model.add_regressor('is_weekend_Weekend')
model.add_regressor('weather_code_Clear')
model.add_regressor('weather_code_Cloudy')
model.add_regressor('weather_code_Rain')
model.add_regressor('weather_code_Scattered_Clouds')
model.add_regressor('weather_code_Snowfall')
model.add_regressor('season_Summer')
model.add_regressor('season_Winter')

model.fit(pr_train)

#Predictions
future = model.make_future_dataframe(periods=len(pr_test), freq='D')
future = future.merge(pr_test[['ds', 't2', 'hum', 'wind_speed', 'is_holiday_No_Holiday', 'is_weekend_Weekend', 'weather_code_Clear', 'weather_code_Cloudy', 'weather_code_Rain', 'weather_code_Scattered_Clouds', 'weather_code_Snowfall', 'season_Summer', 'season_Winter']], on='ds', how='left')
future.dropna(inplace=True)
forecast = model.predict(future)

#MAE, RMSE, R2
mae = np.mean(np.abs(pr_test['y'] - forecast['yhat']))
rmse = np.sqrt(np.mean((pr_test['y'] - forecast['yhat'])**2))
r2 = 1 - np.sum((pr_test['y'] - forecast['yhat'])**2) / np.sum((pr_test['y'] - pr_test['y'].mean())**2)
print('MAE:', mae)
print('RMSE:', rmse)
print('R2:', r2)

# Random Forest

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor

X_train = train_df.drop('cnt', axis=1)
y_train = train_df['cnt']
X_test = test_df.drop('cnt', axis=1)
y_test = test_df['cnt']

model = RandomForestRegressor(n_estimators=4000, max_depth=10, random_state=42)
model.fit(X_train, y_train)

#MAE, RMSE, R2
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(np.mean((y_test - y_pred)**2))
r2 = model.score(X_test, y_test)
print('MAE:', mae)
print('RMSE:', rmse)
print('R2:', r2)

In [ ]:
#Data frame of pred and actual values
pred_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
#Plot of actual vs predicted values
pred_df = pred_df.reset_index()
fig =plt.figure(figsize=(15, 6))
#Remove period
pred_df['timestamp'] = pred_df['timestamp'].dt.to_timestamp()
plt.plot(pred_df['timestamp'], pred_df['Actual'], label='Actual')
plt.plot(pred_df['timestamp'], pred_df['Predicted'], label='Predicted')
plt.title('Actual vs Predicted')
plt.legend()
plt.show()